<a href="https://colab.research.google.com/github/Karishma-Kuria/DL-Assignment4/blob/main/DL_Assignment4_Part2b.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Part 2b. Do various experiments with weights and biases of hyperparameters in weights and biases of various optimizers, layer depth width, learning rate etc. using Keras**

Ref: https://colab.research.google.com/github/wandb/examples/blob/master/colabs/intro/Intro_to_Weights_%26_Biases_keras.ipynb
https://github.com/wandb/examples/tree/master/colabs/keras


Start by installing the library and logging in to your free account.

In [7]:
!pip install wandb -qqq
import wandb

In [8]:
# Log in to your W&B account
wandb.login()

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


True

## 👟 Run an experiment
1️⃣. **Start a new run** and pass in hyperparameters to track

2️⃣. **Log metrics** from training or evaluation

3️⃣. **Visualize results** in the dashboard

In [9]:
import random

# Launch 5 simulated experiments
for run in range(5):
  # 1️⃣ Start a new run to track this script
  wandb.init(
      # Set entity to specify your username or team name
      # ex: entity="carey",
      # Set the project where this run will be logged
      project="basic-intro", 
      # Track hyperparameters and run metadata
      config={
      "learning_rate": 0.02,
      "architecture": "CNN",
      "dataset": "CIFAR-100",})
  
  # This simple block simulates a training loop logging metrics
  offset = random.random() / 5
  for ii in range(2, 10):
      acc = 1 - 2 ** -ii - random.random() / ii - offset
      loss = 2 ** -ii + random.random() / ii + offset
      # 2️⃣ Log metrics from your script to W&B
      wandb.log({"acc": acc, "loss": loss})
      
  # Mark the run as finished
  wandb.finish()

accuracy,▁▅▆▇██
epoch,▁▂▄▅▇█
loss,█▆▄▃▂▁
val_accuracy,▁▅▆▇██
val_loss,█▆▄▃▂▁
accuracy,0.74608
best_epoch,5
best_val_loss,0.85364
epoch,5
loss,0.99413
val_accuracy,0.822


acc,▁▅▆▇▇██▇
loss,█▃▂▂▂▁▁▁
acc,0.77876
loss,0.16159


acc,▁▆▅▇▇▇██
loss,█▅▃▃▂▂▁▁
acc,0.86811
loss,0.15499


acc,▁▅▅▆▇██▇
loss,▆█▃▂▁▁▂▂
acc,0.89787
loss,0.13859


acc,▁▁▃▄▆█▇▆
loss,█▅▃▅▃▂▁▁
acc,0.7366
loss,0.17112


acc,▁▅▅▇▆▇▇█
loss,█▄▃▃▂▁▁▁
acc,0.85043
loss,0.22874


You can find your interactive dashboard by clicking any of the
[project page]()
(for a project-level dashboard)
or [run page]()
(for a run-level dashboard)
links printed by the cell above.

You can also
[interact with W&B dashboards inside Jupyter](http://wandb.me/jupyter-interact-colab).

Here's a dashboard from an autonomous driving project, rendered interactively inside this notebook using the `%wandb` magic:

In [10]:
%wandb stacey/deep-drive/runs/1wyssjcx -h 720

# 🧠 Example Model
Train a quick model in Keras and track results with W&B.

## Install, Import, and Log In

In [11]:
import os
import random

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.datasets import cifar10

# Set the random seeds
os.environ['TF_CUDNN_DETERMINISTIC'] = '1' 
random.seed(hash("setting random seeds") % 2**32 - 1)
np.random.seed(hash("improves reproducibility") % 2**32 - 1)
tf.random.set_seed(hash("by removing stochasticity") % 2**32 - 1)

### Step 0: Install W&B

In [12]:
%%capture
!pip install wandb

### Step 1: Import W&B and Login

In [13]:
import wandb
from wandb.keras import WandbCallback

wandb.login()

True

> Side note: If this is your first time using W&B or you are not logged in, the link that appears after running `wandb.login` will take you to sign-up/login page. Signing up is easy!

## Download and Prepare the Dataset


In [14]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# Subsetting train data and normalizing to [0., 1.]
x_train, x_test = x_train[::5] / 255., x_test / 255.
y_train = y_train[::5]

CLASS_NAMES = ["airplane", "automobile", "bird", "cat",
               "deer", "dog", "frog", "horse", "ship", "truck"]

print('Shape of x_train: ', x_train.shape)
print('Shape of y_train: ', y_train.shape)
print('Shape of x_test: ', x_test.shape)
print('Shape of y_test: ', y_test.shape)

170508288/170498071 [==============================] - 11s 0us/step
Shape of x_train:  (10000, 32, 32, 3)
Shape of y_train:  (10000, 1)
Shape of x_test:  (10000, 32, 32, 3)
Shape of y_test:  (10000, 1)


## Define the Model

Here, we define a standard CNN (with convolution and max-pooling) in Keras.

In [15]:
def Model():
  inputs = keras.layers.Input(shape=(32, 32, 3))

  x = keras.layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu')(inputs)
  x = keras.layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu')(x)
  x = keras.layers.MaxPooling2D(pool_size=2)(x)

  x = keras.layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu')(x)
  x = keras.layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu')(x)

  x = keras.layers.GlobalAveragePooling2D()(x)

  x = keras.layers.Dense(128, activation='relu')(x)
  x = keras.layers.Dense(32, activation='relu')(x)
  
  outputs = keras.layers.Dense(len(CLASS_NAMES), activation='softmax')(x)

  return keras.models.Model(inputs=inputs, outputs=outputs)

## Train the Model

### Step 2: Give `wandb.init` your `config`

You first initialize your wandb run, letting us know some training is about to happen. [Check out the official documentation for `.init` here $\rightarrow$](https://docs.wandb.com/library/init)

That's when you need to set your hyperparameters.
They're passed in as a dictionary via the `config` argument,
and then become available as the `config` attribute of `wandb`.

Learn more about `config` in this [Colab Notebook $\rightarrow$](http://wandb.me/config-colab)

In [16]:
# Initialize wandb with your project name
run = wandb.init(project='my-keras-integration',
                 config={  # and include hyperparameters and metadata
                     "learning_rate": 0.005,
                     "epochs": 5,
                     "batch_size": 1024,
                     "loss_function": "sparse_categorical_crossentropy",
                     "architecture": "CNN",
                     "dataset": "CIFAR-10"
                 })
config = wandb.config  # We'll use this to configure our experiment

# Initialize model like you usually do.
tf.keras.backend.clear_session()
model = Model()
model.summary()

# Compile model like you usually do.
# Notice that we use config, so our metadata matches what gets executed
optimizer = tf.keras.optimizers.Adam(config.learning_rate) 
model.compile(optimizer, config.loss_function, metrics=['acc'])

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 conv2d (Conv2D)             (None, 30, 30, 32)        896       
                                                                 
 conv2d_1 (Conv2D)           (None, 28, 28, 32)        9248      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 14, 14, 32)       0         
 )                                                               
                                                                 
 conv2d_2 (Conv2D)           (None, 12, 12, 32)        9248      
                                                                 
 conv2d_3 (Conv2D)           (None, 10, 10, 32)        9248      
                                                             

### Step 3: Pass `WandbCallback` to `model.fit`

Keras has a [robust callbacks system](https://keras.io/api/callbacks/) that
allows users to separate model definition and the core training logic
from other behaviors that occur during training and testing.

That includes, for example, 

**Click on the Project page link above to see your results!**

In [17]:
# We train with our beloved model.fit
# Notice WandbCallback is used as a regular callback
# We again use config
_ = model.fit(x_train, y_train,
          epochs=config.epochs, 
          batch_size=config.batch_size,
          validation_data=(x_test, y_test),
          callbacks=[WandbCallback()])

Epoch 1/5
10/10 [==============================] - 33s 3s/step - loss: 2.2940 - acc: 0.1173 - val_loss: 2.2962 - val_acc: 0.1064 - _timestamp: 1649934653.0000 - _runtime: 39.0000
Epoch 2/5
10/10 [==============================] - 29s 3s/step - loss: 2.2539 - acc: 0.1357 - val_loss: 2.1813 - val_acc: 0.1523 - _timestamp: 1649934681.0000 - _runtime: 67.0000
Epoch 3/5
10/10 [==============================] - 29s 3s/step - loss: 2.1558 - acc: 0.1826 - val_loss: 2.1763 - val_acc: 0.1739 - _timestamp: 1649934711.0000 - _runtime: 97.0000
Epoch 4/5
10/10 [==============================] - 32s 3s/step - loss: 2.1070 - acc: 0.1917 - val_loss: 2.0553 - val_acc: 0.2052 - _timestamp: 1649934742.0000 - _runtime: 128.0000
Epoch 5/5
10/10 [==============================] - 28s 3s/step - loss: 1.9991 - acc: 0.2120 - val_loss: 1.9776 - val_acc: 0.2297 - _timestamp: 1649934771.0000 - _runtime: 157.0000


## Use `wandb.log` for custom metrics

Here, we log the error rate on the test set.

In [18]:
loss, accuracy = model.evaluate(x_test, y_test)
print('Test Error Rate: ', round((1 - accuracy) * 100, 2))

# With wandb.log, we can easily pass in metrics as key-value pairs.
wandb.log({'Test Error Rate': round((1 - accuracy) * 100, 2)})

run.finish()

313/313 [==============================] - 7s 22ms/step - loss: 1.9776 - acc: 0.2297
Test Error Rate:  77.03



Test Error Rate,▁
acc,▁▂▆▆█
epoch,▁▃▅▆█
loss,█▇▅▄▁
val_acc,▁▄▅▇█
val_loss,█▅▅▃▁
Test Error Rate,77.03
acc,0.212
best_epoch,4
best_val_loss,1.97759
epoch,4


## Log predictions on test data with `WandbCallback`

When machine learning models break, they usually don't throw errors. They fail silently, and the only way to detect them is to look at their behavior on examples. Without an experiment-tracking library, this requires writing a lot of extra code. That's time you're not spending training machines!

In this section, we demonstrate how this is easily done with the `WandbCallback` by logging both predictions on the test set and their matching images.

Let's take this opportunity to do a little experiment as well. We'll

* change learning rate from `0.005` to `0.001`.
* change batch size from `1024` to `32`.

In [19]:
# Initialize wandb with your project name
run = wandb.init(project='my-keras-integration',
                 config={  # and include hyperparameters and metadata
                     "learning_rate": 0.001,
                     "epochs": 5,
                     "batch_size": 32,
                     "loss_function": "sparse_categorical_crossentropy",
                     "architecture": "CNN",
                     "dataset": "CIFAR-10"
                 })
config = wandb.config  # We'll use this to configure our experiment

# Initialize model like you usually do.
tf.keras.backend.clear_session()
model = Model()
model.summary()

# Compile model like you usually do.
optimizer = tf.keras.optimizers.Adam(config.learning_rate) 
model.compile(optimizer, config.loss_function, metrics=['acc'])

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 conv2d (Conv2D)             (None, 30, 30, 32)        896       
                                                                 
 conv2d_1 (Conv2D)           (None, 28, 28, 32)        9248      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 14, 14, 32)       0         
 )                                                               
                                                                 
 conv2d_2 (Conv2D)           (None, 12, 12, 32)        9248      
                                                                 
 conv2d_3 (Conv2D)           (None, 10, 10, 32)        9248      
                                                             

In [20]:
# We focus on a subset of images, since this is for human review
val_images, val_labels = x_test[:32], y_test[:32]

# Our beloved model.fit returns
# By passing arguments to WandbCallback, we change its behavior
_ = model.fit(x_train, y_train,
              epochs=config.epochs, 
              batch_size=config.batch_size,
              validation_data=(x_test, y_test),
              callbacks=[WandbCallback(data_type='image', 
                                       validation_data=(val_images, val_labels), 
                                       labels=CLASS_NAMES)])

run.finish()

wandb: WARNING The data_type argument of wandb.keras.WandbCallback is deprecated and will be removed in a future release. Please use input_type instead.
wandb: WARNING Setting input_type = data_type.


Epoch 1/5
313/313 [==============================] - 36s 114ms/step - loss: 2.0602 - acc: 0.1992 - val_loss: 1.8279 - val_acc: 0.2757 - _timestamp: 1649934877.0000 - _runtime: 41.0000
Epoch 2/5
313/313 [==============================] - 37s 117ms/step - loss: 1.7893 - acc: 0.3004 - val_loss: 1.7227 - val_acc: 0.3258 - _timestamp: 1649934914.0000 - _runtime: 78.0000
Epoch 3/5
313/313 [==============================] - 37s 117ms/step - loss: 1.7076 - acc: 0.3276 - val_loss: 1.6728 - val_acc: 0.3552 - _timestamp: 1649934951.0000 - _runtime: 115.0000
Epoch 4/5
313/313 [==============================] - 36s 115ms/step - loss: 1.6496 - acc: 0.3642 - val_loss: 1.6032 - val_acc: 0.3784 - _timestamp: 1649934987.0000 - _runtime: 151.0000
Epoch 5/5
313/313 [==============================] - 35s 113ms/step - loss: 1.5854 - acc: 0.3887 - val_loss: 1.6568 - val_acc: 0.3542 - _timestamp: 1649935022.0000 - _runtime: 186.0000



acc,▁▅▆▇█
epoch,▁▃▅▆█
loss,█▄▃▂▁
val_acc,▁▄▆█▆
val_loss,█▅▃▁▃
acc,0.3887
best_epoch,3
best_val_loss,1.6032
epoch,4
loss,1.58538
val_acc,0.3542


## 🥕 Simple Keras DNN
Run this model to train a simple MNIST classifier, and click on the project page link to see your results stream in live to a W&B project.



In addition to logged [metrics](https://docs.wandb.ai/ref/app/pages/run-page#charts-tab),
[system information](https://docs.wandb.ai/ref/app/pages/run-page#system-tab),
[hyperparameters](https://docs.wandb.ai/ref/app/pages/run-page#overview-tab), and
[terminal output](https://docs.wandb.ai/ref/app/pages/run-page#logs-tab),
you'll see an [interactive table](https://docs.wandb.ai/guides/data-vis)
with model inputs and outputs.

See [this Report](http://wandb.me/debug-dashboard)
for an example visualization backed by this table --
a model debugging dashboard --
or read more about our Tables feature [here](https://docs.wandb.ai/guides/data-vis/tables-quickstart).

In [21]:
import random

import numpy as np
import tensorflow as tf
from wandb.keras import WandbCallback

# Simple Keras Model

# Launch 20 experiments, trying different dropout rates
for run in range(20):
  # Start a run, tracking hyperparameters
  wandb.init(
      project="keras-intro",
      # Set entity to specify your username or team name
      # ex: entity="wandb",
      config={
          "layer_1": 512,
          "activation_1": "relu",
          "dropout": random.uniform(0.01, 0.80),
          "layer_2": 10,
          "activation_2": "softmax",
          "optimizer": "sgd",
          "loss": "sparse_categorical_crossentropy",
          "metric": "accuracy",
          "epoch": 6,
          "batch_size": 256
      })
  config = wandb.config

  # Get the data
  mnist = tf.keras.datasets.mnist
  (x_train, y_train), (x_test, y_test) = mnist.load_data()
  x_train, x_test = x_train / 255.0, x_test / 255.0
  x_train, y_train = x_train[::5], y_train[::5]  # Subset data for a faster demo
  x_test, y_test = x_test[::20], y_test[::20]
  labels = [str(digit) for digit in range(np.max(y_train) + 1)]

  # Build a model
  model = tf.keras.models.Sequential([
      tf.keras.layers.Flatten(input_shape=(28, 28)),
      tf.keras.layers.Dense(config.layer_1, activation=config.activation_1),
      tf.keras.layers.Dropout(config.dropout),
      tf.keras.layers.Dense(config.layer_2, activation=config.activation_2)
      ])

  model.compile(optimizer=config.optimizer,
                loss=config.loss,
                metrics=[config.metric]
                )

  # WandbCallback auto-saves all metrics from model.fit(), plus predictions on validation_data
  logging_callback = WandbCallback(log_evaluation=True)

  history = model.fit(x=x_train, y=y_train,
                      epochs=config.epoch,
                      batch_size=config.batch_size,
                      validation_data=(x_test, y_test),
                      callbacks=[logging_callback]
                      )

  # Mark the run as finished
  wandb.finish()

Epoch 1/6
47/47 [==============================] - 1s 20ms/step - loss: 2.1653 - accuracy: 0.2253 - val_loss: 1.8586 - val_accuracy: 0.5860 - _timestamp: 1649935061.0000 - _runtime: 11.0000
Epoch 2/6
47/47 [==============================] - 1s 16ms/step - loss: 1.7594 - accuracy: 0.4782 - val_loss: 1.5261 - val_accuracy: 0.6840 - _timestamp: 1649935061.0000 - _runtime: 11.0000
Epoch 3/6
47/47 [==============================] - 1s 17ms/step - loss: 1.4737 - accuracy: 0.6080 - val_loss: 1.2776 - val_accuracy: 0.7320 - _timestamp: 1649935062.0000 - _runtime: 12.0000
Epoch 4/6
47/47 [==============================] - 1s 16ms/step - loss: 1.2710 - accuracy: 0.6687 - val_loss: 1.0994 - val_accuracy: 0.7700 - _timestamp: 1649935063.0000 - _runtime: 13.0000
Epoch 5/6
47/47 [==============================] - 1s 16ms/step - loss: 1.1200 - accuracy: 0.7113 - val_loss: 0.9676 - val_accuracy: 0.8000 - _timestamp: 1649935064.0000 - _runtime: 14.0000
Epoch 6/6
47/47 [==============================] -

accuracy,▁▄▆▇██
epoch,▁▂▄▅▇█
loss,█▆▄▃▂▁
val_accuracy,▁▄▆▇██
val_loss,█▆▄▃▂▁
accuracy,0.73658
best_epoch,5
best_val_loss,0.87009
epoch,5
loss,1.00706
val_accuracy,0.81


Epoch 1/6
47/47 [==============================] - 1s 20ms/step - loss: 2.1145 - accuracy: 0.2584 - val_loss: 1.8027 - val_accuracy: 0.5980 - _timestamp: 1649935086.0000 - _runtime: 10.0000
Epoch 2/6
47/47 [==============================] - 1s 17ms/step - loss: 1.7262 - accuracy: 0.4812 - val_loss: 1.4788 - val_accuracy: 0.7060 - _timestamp: 1649935087.0000 - _runtime: 11.0000
Epoch 3/6
47/47 [==============================] - 1s 16ms/step - loss: 1.4555 - accuracy: 0.5940 - val_loss: 1.2450 - val_accuracy: 0.7560 - _timestamp: 1649935088.0000 - _runtime: 12.0000
Epoch 4/6
47/47 [==============================] - 1s 16ms/step - loss: 1.2606 - accuracy: 0.6604 - val_loss: 1.0766 - val_accuracy: 0.7820 - _timestamp: 1649935088.0000 - _runtime: 12.0000
Epoch 5/6
47/47 [==============================] - 1s 17ms/step - loss: 1.1186 - accuracy: 0.7013 - val_loss: 0.9529 - val_accuracy: 0.8040 - _timestamp: 1649935089.0000 - _runtime: 13.0000
Epoch 6/6
47/47 [==============================] -

accuracy,▁▄▆▇██
epoch,▁▂▄▅▇█
loss,█▆▄▃▂▁
val_accuracy,▁▅▆▇██
val_loss,█▆▄▃▂▁
accuracy,0.72292
best_epoch,5
best_val_loss,0.85996
epoch,5
loss,1.02357
val_accuracy,0.81


Epoch 1/6
47/47 [==============================] - 1s 20ms/step - loss: 2.1734 - accuracy: 0.2292 - val_loss: 1.8634 - val_accuracy: 0.5680 - _timestamp: 1649935109.0000 - _runtime: 8.0000
Epoch 2/6
47/47 [==============================] - 1s 17ms/step - loss: 1.7539 - accuracy: 0.4892 - val_loss: 1.5233 - val_accuracy: 0.7160 - _timestamp: 1649935110.0000 - _runtime: 9.0000
Epoch 3/6
47/47 [==============================] - 1s 16ms/step - loss: 1.4770 - accuracy: 0.6091 - val_loss: 1.2766 - val_accuracy: 0.7600 - _timestamp: 1649935110.0000 - _runtime: 9.0000
Epoch 4/6
47/47 [==============================] - 1s 17ms/step - loss: 1.2648 - accuracy: 0.6773 - val_loss: 1.0984 - val_accuracy: 0.7800 - _timestamp: 1649935111.0000 - _runtime: 10.0000
Epoch 5/6
47/47 [==============================] - 1s 16ms/step - loss: 1.1167 - accuracy: 0.7143 - val_loss: 0.9693 - val_accuracy: 0.7980 - _timestamp: 1649935112.0000 - _runtime: 11.0000
Epoch 6/6
47/47 [==============================] - 1s

...

accuracy,▁▅▇▇██
epoch,▁▂▄▅▇█
loss,█▆▄▃▂▁
val_accuracy,▁▅▆▇██
val_loss,█▆▄▃▂▁
accuracy,0.76983
best_epoch,5
best_val_loss,0.84821
epoch,5
loss,0.9489
val_accuracy,0.8


Epoch 1/6
47/47 [==============================] - 1s 21ms/step - loss: 2.1617 - accuracy: 0.2314 - val_loss: 1.8761 - val_accuracy: 0.5860 - _timestamp: 1649935469.0000 - _runtime: 8.0000
Epoch 2/6
47/47 [==============================] - 1s 18ms/step - loss: 1.7439 - accuracy: 0.5241 - val_loss: 1.5210 - val_accuracy: 0.7000 - _timestamp: 1649935470.0000 - _runtime: 9.0000
Epoch 3/6
47/47 [==============================] - 1s 18ms/step - loss: 1.4473 - accuracy: 0.6445 - val_loss: 1.2636 - val_accuracy: 0.7540 - _timestamp: 1649935471.0000 - _runtime: 10.0000
Epoch 4/6
47/47 [==============================] - 1s 17ms/step - loss: 1.2311 - accuracy: 0.7011 - val_loss: 1.0819 - val_accuracy: 0.7860 - _timestamp: 1649935472.0000 - _runtime: 11.0000
Epoch 5/6
47/47 [==============================] - 1s 17ms/step - loss: 1.0826 - accuracy: 0.7329 - val_loss: 0.9504 - val_accuracy: 0.7960 - _timestamp: 1649935472.0000 - _runtime: 11.0000
Epoch 6/6
47/47 [==============================] - 1

accuracy,▁▅▆▇██
epoch,▁▂▄▅▇█
loss,█▆▄▃▂▁
val_accuracy,▁▅▆▇██
val_loss,█▆▄▃▂▁
accuracy,0.76133
best_epoch,5
best_val_loss,0.85185
epoch,5
loss,0.96277
val_accuracy,0.81


Epoch 1/6
47/47 [==============================] - 1s 21ms/step - loss: 2.1260 - accuracy: 0.2860 - val_loss: 1.8780 - val_accuracy: 0.5720 - _timestamp: 1649935490.0000 - _runtime: 8.0000
Epoch 2/6
47/47 [==============================] - 1s 18ms/step - loss: 1.7119 - accuracy: 0.6110 - val_loss: 1.5199 - val_accuracy: 0.7100 - _timestamp: 1649935491.0000 - _runtime: 9.0000
Epoch 3/6
47/47 [==============================] - 1s 18ms/step - loss: 1.4043 - accuracy: 0.7145 - val_loss: 1.2555 - val_accuracy: 0.7640 - _timestamp: 1649935492.0000 - _runtime: 10.0000
Epoch 4/6
47/47 [==============================] - 1s 16ms/step - loss: 1.1807 - accuracy: 0.7614 - val_loss: 1.0693 - val_accuracy: 0.7800 - _timestamp: 1649935493.0000 - _runtime: 11.0000
Epoch 5/6
47/47 [==============================] - 1s 16ms/step - loss: 1.0223 - accuracy: 0.7878 - val_loss: 0.9376 - val_accuracy: 0.8020 - _timestamp: 1649935494.0000 - _runtime: 12.0000
Epoch 6/6
47/47 [==============================] - 1

accuracy,▁▅▇▇██
epoch,▁▂▄▅▇█
loss,█▆▄▃▂▁
val_accuracy,▁▅▇▇██
val_loss,█▆▄▃▂▁
accuracy,0.8025
best_epoch,5
best_val_loss,0.83944
epoch,5
loss,0.91042
val_accuracy,0.808


## 📊 Visualize Results
The results can be seen here: https://wandb.ai/karishma/keras-intro/runs/2g2m398k?workspace=user-karishma

